In [1]:
using DataFramesMeta
using StatsBase
using CSV

dir = "/Users/fujisetakumi/_private/data_science/100knocks-preprocess/data/"
df_customers  = CSV.read(dir * "customers.csv", DataFrame)
df_categories = CSV.read(dir * "categories.csv", DataFrame)
df_products   = CSV.read(dir * "products.csv", DataFrame)
df_geocodes   = CSV.read(dir * "geocodes.csv", DataFrame)
df_receipts   = CSV.read(dir * "receipts.csv", DataFrame)
df_stores     = CSV.read(dir * "stores.csv", DataFrame)
;
ENV["COLUMNS"] = 1000
;

In [2]:
# P-051 エポック秒から日だけ取り出して0埋め
using Dates
@chain df_receipts begin
    @rtransform(:sales_date = Date(unix2datetime(:sales_epoch)))
    @rtransform(:sales_day = string(Dates.day(:sales_date), pad = 2))
    select(:receipt_no, :receipt_sub_no, :sales_day)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_day
,Int64,Int64,String
1,112,1,03
2,1132,2,18
3,1102,1,12
4,1132,1,05
5,1102,2,21
6,1112,1,05
7,1102,2,05
8,1102,1,22
9,1112,2,04


In [3]:
# P-052 売上金額が2000円以上か否かで2値化
@chain df_receipts begin
    @rsubset(occursin(r"^[^Z]", :customer_id))
    groupby(:customer_id)
    combine(:amount => sum)
    @rtransform(:gt2000 = :amount_sum > 2000 ? 1 : 0)
    first(10)
end

Row,customer_id,amount_sum,gt2000
,String15,Int64,Int64
1,CS006214000001,7364,1
2,CS008415000097,1895,0
3,CS028414000014,6222,1
4,CS025415000050,5736,1
5,CS003515000195,5412,1
6,CS024514000042,533,0
7,CS040415000178,6149,1
8,CS027514000015,2788,1
9,CS025415000134,4902,1


In [4]:
# P-053
@chain df_customers begin
    @rtransform(:tokyo = convert(Int, 
        100 <= parse(Int, :postal_cd[1:3]) <= 209))
    # unique で行数が増えないようにする
    innerjoin(unique(df_receipts, :customer_id), on= :customer_id => :customer_id)
    groupby(:tokyo)
    combine(:tokyo => length)
end

Row,tokyo,tokyo_length
,Int64,Int64
1,0,3906
2,1,4400


In [5]:
# P-054
@chain df_customers begin
    @rtransform(:prefecture_cd = @chain :address begin
            if occursin(r"埼玉県", _)
                11
            elseif occursin(r"千葉県", _)
                12
            elseif occursin(r"東京都", _)
                13
            elseif occursin(r"神奈川県", _)
                14
            end
        end)
    select(:customer_id, :address, :prefecture_cd)
    first(10)
end

Row,customer_id,address,prefecture_cd
,String15,String,Int64
1,CS021313000114,神奈川県伊勢原市粟窪**********,14
2,CS037613000071,東京都江東区南砂**********,13
3,CS031415000172,東京都渋谷区代々木**********,13
4,CS028811000001,神奈川県横浜市泉区和泉町**********,14
5,CS001215000145,東京都大田区仲六郷**********,13
6,CS020401000016,東京都板橋区若木**********,13
7,CS015414000103,東京都江東区北砂**********,13
8,CS029403000008,千葉県浦安市海楽**********,12
9,CS015804000004,東京都江東区北砂**********,13


In [6]:
# P-055 合計金額の四分位点
stats = @chain df_receipts begin
    groupby(:customer_id)
    combine(:amount => sum)
    sort(:amount_sum)
end
_, pct25, pct50, pct75 = nquantile(stats.amount_sum, 4)
@chain stats begin
    @rtransform(:rank = @chain :amount_sum begin
            if _ < pct25
                1
            elseif _ < pct50
                2
            elseif _ < pct75
                3
            else
                4
            end
        end)
    sort(:customer_id)
end

Row,customer_id,amount_sum,rank
,String15,Int64,Int64
1,CS001113000004,1298,2
2,CS001114000005,626,2
3,CS001115000010,3044,3
4,CS001205000004,1988,3
5,CS001205000006,3337,3
6,CS001211000025,456,1
7,CS001212000027,448,1
8,CS001212000031,296,1
9,CS001212000046,228,1


In [7]:
# P-056 年代
@chain df_customers begin
    @rtransform(:generation = @chain floor(Int, :age / 10) * 10 _ > 60 ? 60 : _)
    select(:customer_id, :birth_day, :generation)
    first(10)
end

Row,customer_id,birth_day,generation
,String15,Date,Int64
1,CS021313000114,1981-04-29,30
2,CS037613000071,1952-04-01,60
3,CS031415000172,1976-10-04,40
4,CS028811000001,1933-03-27,60
5,CS001215000145,1995-03-29,20
6,CS020401000016,1974-09-15,40
7,CS015414000103,1977-08-09,40
8,CS029403000008,1973-08-17,40
9,CS015804000004,1931-05-02,60


In [8]:
# P-057
@chain df_customers begin
    @rtransform(:generation = @chain floor(Int, :age / 10) * 10 _ > 60 ? 60 : _)
    select(:gender_cd, :generation)
    sort([:gender_cd, :generation])
    @rtransform(:cd = string(:gender_cd, "g", :generation))
    unique
end

Row,gender_cd,generation,cd
,Int64,Int64,String
1,0,10,0g10
2,0,20,0g20
3,0,30,0g30
4,0,40,0g40
5,0,50,0g50
6,0,60,0g60
7,1,10,1g10
8,1,20,1g20
9,1,30,1g30


In [9]:
# P-058 性別を横持ちにする(ダミー変数化。one-hot encoding)
# 参考: https://yjunechoe.github.io/posts/2022-11-13-dataframes-jl-and-accessories/
@chain df_customers begin
    select(:customer_id, :gender_cd)
    @aside gender_cds = unique(df_customers.gender_cd) |> sort
    transform(:gender_cd => ByRow(v -> gender_cds .== v)
        => Symbol.(:gender_cd_, gender_cds))
    select(Not(:gender_cd))
    first(10)
end

Row,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
,String15,Bool,Bool,Bool
1,CS021313000114,false,true,false
2,CS037613000071,false,false,true
3,CS031415000172,false,true,false
4,CS028811000001,false,true,false
5,CS001215000145,false,true,false
6,CS020401000016,true,false,false
7,CS015414000103,false,true,false
8,CS029403000008,true,false,false
9,CS015804000004,true,false,false


In [10]:
# P-059 平均０、標準偏差１に標準化(Zスコア)
@chain df_receipts begin
    @rsubset(occursin(r"^[^Z]", :customer_id))
    groupby(:customer_id)
    combine(:amount => sum)
    @transform(:amount_sum_std = zscore(:amount_sum))
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum,amount_sum_std
,String15,Int64,Float64
1,CS001113000004,1298,-0.45935
2,CS001114000005,626,-0.706348
3,CS001115000010,3044,0.182403
4,CS001205000004,1988,-0.205737
5,CS001205000006,3337,0.290096
6,CS001211000025,456,-0.768832
7,CS001212000027,448,-0.771773
8,CS001212000031,296,-0.827641
9,CS001212000046,228,-0.852635


In [11]:
# P-060 最小値0、最大値1に正規化
@chain df_receipts begin
    @rsubset(occursin(r"^[^Z]", :customer_id))
    groupby(:customer_id)
    combine(:amount => sum)
    @aside min_amount = minimum(_.amount_sum)
    @aside max_amount = maximum(_.amount_sum)
    @rtransform(:amount_scale = 
        (:amount_sum - min_amount) / (max_amount - min_amount))
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum,amount_scale
,String15,Int64,Float64
1,CS001113000004,1298,0.0533542
2,CS001114000005,626,0.0241571
3,CS001115000010,3044,0.129214
4,CS001205000004,1988,0.0833333
5,CS001205000006,3337,0.141945
6,CS001211000025,456,0.0167709
7,CS001212000027,448,0.0164234
8,CS001212000031,296,0.00981926
9,CS001212000046,228,0.00686479


In [12]:
# P-061 常用対数化
@chain df_receipts begin
    @subset(occursin.(r"^[^Z]", :customer_id))
    groupby(:customer_id)
    combine(:amount => sum)
    @transform(:amount_sum_log10 = log10.(:amount_sum))
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum,amount_sum_log10
,String15,Int64,Float64
1,CS001113000004,1298,3.11327
2,CS001114000005,626,2.79657
3,CS001115000010,3044,3.48344
4,CS001205000004,1988,3.29842
5,CS001205000006,3337,3.52336
6,CS001211000025,456,2.65896
7,CS001212000027,448,2.65128
8,CS001212000031,296,2.47129
9,CS001212000046,228,2.35793


In [13]:
# P-062 自然対数化
@chain df_receipts begin
    @rsubset(occursin(r"^[^Z]", :customer_id))
    groupby(:customer_id)
    combine(:amount => sum)
    @rtransform(:amount_sum_log = log(:amount_sum))
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum,amount_sum_log
,String15,Int64,Float64
1,CS001113000004,1298,7.16858
2,CS001114000005,626,6.43935
3,CS001115000010,3044,8.02093
4,CS001205000004,1988,7.59488
5,CS001205000006,3337,8.11283
6,CS001211000025,456,6.12249
7,CS001212000027,448,6.10479
8,CS001212000031,296,5.69036
9,CS001212000046,228,5.42935


In [14]:
# P-063 各商品の利益額
@chain df_products begin
    select(:product_cd, :unit_price, :unit_cost)
    @rtransform(:unit_profit = :unit_price - :unit_cost)
    sort(:product_cd)
    first(10)
end

Row,product_cd,unit_price,unit_cost,unit_profit
,String15,Int64?,Int64?,Int64?
1,P040101001,198,149,49
2,P040101002,218,164,54
3,P040101003,230,173,57
4,P040101004,248,186,62
5,P040101005,268,201,67
6,P040101006,298,224,74
7,P040101007,338,254,84
8,P040101008,420,315,105
9,P040101009,498,374,124


In [15]:
# P-064 利益率の全体平均
@chain df_products begin
    select(:product_cd, :unit_price, :unit_cost)
    @rtransform(:unit_profit_ratio = 
        (:unit_price - :unit_cost) / :unit_price)
    # @subset(ismissing.(:unit_profit_ratio)) # =>b 7件存在
    dropmissing(:unit_profit_ratio)
    combine(:unit_profit_ratio => mean)
end

Row,unit_profit_ratio_mean
,Float64
1,0.249114


In [16]:
# P-065 利益率３０％になる単価: 切り捨て
@chain df_products begin
    select(:product_cd, :unit_cost)
    @rtransform(:unit_new_price = 
        ismissing(:unit_cost) ? 0 : floor(Int, :unit_cost / 0.7))
    @rtransform(:unit_new_profit = 
        (:unit_new_price - :unit_cost) / :unit_new_price)
    sort(:product_cd)
    first(10)
end

Row,product_cd,unit_cost,unit_new_price,unit_new_profit
,String15,Int64?,Int64,Float64?
1,P040101001,149,212,0.29717
2,P040101002,164,234,0.299145
3,P040101003,173,247,0.299595
4,P040101004,186,265,0.298113
5,P040101005,201,287,0.299652
6,P040101006,224,320,0.3
7,P040101007,254,362,0.298343
8,P040101008,315,450,0.3
9,P040101009,374,534,0.299625


In [17]:
# P-066 利益率３０％になる単価: 丸め
@chain df_products begin
    select(:product_cd, :unit_cost)
    @rtransform(:unit_new_price =
        ismissing(:unit_cost) ? 0 : round(Int, :unit_cost / 0.7))
    @rtransform(:unit_new_profit = 
        (:unit_new_price - :unit_cost) / :unit_new_price)
    sort(:product_cd)
    first(10)
end

Row,product_cd,unit_cost,unit_new_price,unit_new_profit
,String15,Int64?,Int64,Float64?
1,P040101001,149,213,0.300469
2,P040101002,164,234,0.299145
3,P040101003,173,247,0.299595
4,P040101004,186,266,0.300752
5,P040101005,201,287,0.299652
6,P040101006,224,320,0.3
7,P040101007,254,363,0.300275
8,P040101008,315,450,0.3
9,P040101009,374,534,0.299625


In [18]:
# P-066 利益率３０％になる単価: 切り上げ
@chain df_products begin
    select(:product_cd, :unit_cost)
    @rtransform(:unit_new_price =
        ismissing(:unit_cost) ? 0 : ceil(Int, :unit_cost / 0.7))
    @rtransform(:unit_new_profit = 
        (:unit_new_price - :unit_cost) / :unit_new_price)
    sort(:product_cd)
    first(10)
end

Row,product_cd,unit_cost,unit_new_price,unit_new_profit
,String15,Int64?,Int64,Float64?
1,P040101001,149,213,0.300469
2,P040101002,164,235,0.302128
3,P040101003,173,248,0.302419
4,P040101004,186,266,0.300752
5,P040101005,201,288,0.302083
6,P040101006,224,320,0.3
7,P040101007,254,363,0.300275
8,P040101008,315,451,0.301552
9,P040101009,374,535,0.300935


In [19]:
# P-068 消費税
@chain df_products begin
    select(:product_cd, :unit_price)
    @rtransform(:tax_price = ismissing(:unit_price) ? missing
        : floor(Int, :unit_price * 1.1))
    sort(:product_cd)
    first(10)
end

Row,product_cd,unit_price,tax_price
,String15,Int64?,Int64?
1,P040101001,198,217
2,P040101002,218,239
3,P040101003,230,253
4,P040101004,248,272
5,P040101005,268,294
6,P040101006,298,327
7,P040101007,338,371
8,P040101008,420,462
9,P040101009,498,547


In [20]:
# P-069 売上金額合計のうち瓶詰缶詰の比率
categories_amounts = @chain df_receipts begin
    innerjoin(df_products, on= :product_cd => :product_cd,
        makeunique=true)
    groupby([:customer_id, :category_major_cd])
    combine(:amount => sum)
    sort([:customer_id, :category_major_cd])
end

customers_amounts = @chain categories_amounts begin
    groupby(:customer_id)
    combine(:amount_sum => sum => :all_amount_sum)
end

@chain categories_amounts begin
    @rsubset(:category_major_cd == 7)
    innerjoin(customers_amounts, on= :customer_id => :customer_id)
    @rtransform(:amount_ratio = :amount_sum / :all_amount_sum)
    sort(:customer_id)
    first(10)
end

Row,customer_id,category_major_cd,amount_sum,all_amount_sum,amount_ratio
,String15,Int64,Int64,Int64,Float64
1,CS001113000004,7,1298,1298,1.0
2,CS001114000005,7,486,626,0.776358
3,CS001115000010,7,2694,3044,0.88502
4,CS001205000004,7,346,1988,0.174044
5,CS001205000006,7,2004,3337,0.600539
6,CS001212000027,7,200,448,0.446429
7,CS001212000031,7,296,296,1.0
8,CS001212000046,7,108,228,0.473684
9,CS001212000070,7,308,456,0.675439


In [21]:
# P-070 経過日数
using Dates
@chain df_receipts begin
    innerjoin(df_customers, on= :customer_id => :customer_id,
        makeunique=true)
    @rtransform(:sales_ymd = Date(string(:sales_ymd), "YYYYmmdd"))
    @rtransform(:application_date = Date(string(:application_date), "YYYYmmdd"))
    select(:customer_id, :sales_ymd, :application_date)
    @rtransform(:elapsed_days = :sales_ymd - :application_date)
    sort(:customer_id)
    first(10)
end

Row,customer_id,sales_ymd,application_date,elapsed_days
,String15,Date,Date,Day
1,CS001113000004,2019-03-08,2015-11-05,1219 days
2,CS001113000004,2019-03-08,2015-11-05,1219 days
3,CS001114000005,2018-05-03,2016-04-12,751 days
4,CS001114000005,2018-05-03,2016-04-12,751 days
5,CS001114000005,2019-07-31,2016-04-12,1205 days
6,CS001114000005,2019-07-31,2016-04-12,1205 days
7,CS001115000010,2019-04-05,2015-04-17,1449 days
8,CS001115000010,2018-07-01,2015-04-17,1171 days
9,CS001115000010,2017-12-28,2015-04-17,986 days


In [22]:
# P-071 経過月数
using Dates
@chain df_receipts begin
    innerjoin(df_customers, on= :customer_id => :customer_id,
        makeunique=true)
    @rtransform(:sales_ymd = Date(string(:sales_ymd), "YYYYmmdd"))
    @rtransform(:application_date = Date(string(:application_date), "YYYYmmdd"))
    select(:customer_id, :sales_ymd, :application_date)
    @rtransform(:elapsed_month = 
        Dates.value(:sales_ymd - :application_date) / 30
        |> floor |> Int)
    sort(:customer_id)
    first(10)
end

Row,customer_id,sales_ymd,application_date,elapsed_month
,String15,Date,Date,Int64
1,CS001113000004,2019-03-08,2015-11-05,40
2,CS001113000004,2019-03-08,2015-11-05,40
3,CS001114000005,2018-05-03,2016-04-12,25
4,CS001114000005,2018-05-03,2016-04-12,25
5,CS001114000005,2019-07-31,2016-04-12,40
6,CS001114000005,2019-07-31,2016-04-12,40
7,CS001115000010,2019-04-05,2015-04-17,48
8,CS001115000010,2018-07-01,2015-04-17,39
9,CS001115000010,2017-12-28,2015-04-17,32


In [23]:
# P-072 経過年数
using Dates
@chain df_receipts begin
    innerjoin(df_customers, on= :customer_id => :customer_id,
        makeunique=true)
    @rtransform(:sales_ymd = Date(string(:sales_ymd), "YYYYmmdd"))
    @rtransform(:application_date = Date(string(:application_date), "YYYYmmdd"))
    select(:customer_id, :sales_ymd, :application_date)
    @rtransform(:elapsed_year = 
        Dates.value(:sales_ymd - :application_date) / 365
        |> floor |> Int)
    sort(:customer_id)
    first(10)
end

Row,customer_id,sales_ymd,application_date,elapsed_year
,String15,Date,Date,Int64
1,CS001113000004,2019-03-08,2015-11-05,3
2,CS001113000004,2019-03-08,2015-11-05,3
3,CS001114000005,2018-05-03,2016-04-12,2
4,CS001114000005,2018-05-03,2016-04-12,2
5,CS001114000005,2019-07-31,2016-04-12,3
6,CS001114000005,2019-07-31,2016-04-12,3
7,CS001115000010,2019-04-05,2015-04-17,3
8,CS001115000010,2018-07-01,2015-04-17,3
9,CS001115000010,2017-12-28,2015-04-17,2


In [24]:
# P-073 エポック秒による経過時間
using Dates
@chain df_receipts begin
    innerjoin(df_customers, on= :customer_id => :customer_id,
        makeunique=true)
    @rtransform(:sales_ymd = Date(string(:sales_ymd), "YYYYmmdd"))
    @rtransform(:application_date = Date(string(:application_date), "YYYYmmdd"))
    select(:customer_id, :sales_ymd, :application_date)
    @rtransform(:elapsed_unixtime = 
        datetime2unix(:sales_ymd |> DateTime) 
        - datetime2unix(:application_date |> DateTime))
    sort(:customer_id)
    first(10)
end

Row,customer_id,sales_ymd,application_date,elapsed_unixtime
,String15,Date,Date,Float64
1,CS001113000004,2019-03-08,2015-11-05,1.05322e8
2,CS001113000004,2019-03-08,2015-11-05,1.05322e8
3,CS001114000005,2018-05-03,2016-04-12,6.48864e7
4,CS001114000005,2018-05-03,2016-04-12,6.48864e7
5,CS001114000005,2019-07-31,2016-04-12,1.04112e8
6,CS001114000005,2019-07-31,2016-04-12,1.04112e8
7,CS001115000010,2019-04-05,2015-04-17,1.25194e8
8,CS001115000010,2018-07-01,2015-04-17,1.01174e8
9,CS001115000010,2017-12-28,2015-04-17,8.51904e7


In [25]:
# P-074 当該週の月曜日からの経過日数
using Dates
@chain df_receipts begin
    select(:receipt_no, :sales_ymd)
    @rtransform(:sales_ymd = Date(string(:sales_ymd), "YYYYmmdd"))
    @rtransform(:dayabbr = dayabbr(:sales_ymd))
    @rtransform(:last_monday = firstdayofweek(:sales_ymd))
    @rtransform(:elapsed_day_of_week = dayofweek(:sales_ymd) - 1)
    first(10)
end

Row,receipt_no,sales_ymd,dayabbr,last_monday,elapsed_day_of_week
,Int64,Date,String,Date,Int64
1,112,2018-11-03,Sat,2018-10-29,5
2,1132,2018-11-18,Sun,2018-11-12,6
3,1102,2017-07-12,Wed,2017-07-10,2
4,1132,2019-02-05,Tue,2019-02-04,1
5,1102,2018-08-21,Tue,2018-08-20,1
6,1112,2019-06-05,Wed,2019-06-03,2
7,1102,2018-12-05,Wed,2018-12-03,2
8,1102,2019-09-22,Sun,2019-09-16,6
9,1112,2017-05-04,Thu,2017-05-01,3


In [26]:
# P-075 ランダムに1%のデータを取得。うち10件を表示
using Random
count = nrow(df_customers)
df_customers[shuffle(1:count)[1:(count ÷ 100)], :] |> 
    df -> first(df, 10)

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS006515000100,天野 明慶,9,不明,1963-08-01,55,224-0065,神奈川県横浜市都筑区高山**********,S14006,20150211,8-20090510-B
2,CS040513000154,諏訪 朝陽,1,女性,1961-07-11,57,226-0016,神奈川県横浜市緑区霧が丘**********,S14040,20150717,C-20091220-A
3,CS016202000003,小川 明慶,0,男性,1990-05-01,28,180-0021,東京都武蔵野市桜堤**********,S13016,20150717,0-00000000-0
4,CS013615000073,寺田 由美子,1,女性,1953-05-12,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150714,0-00000000-0
5,CS026414000062,岡本 ひかり,1,女性,1970-09-27,48,251-0046,神奈川県藤沢市辻堂西海岸**********,S14026,20150610,F-20101013-F
6,CS040413000143,梶原 夏希,1,女性,1973-05-23,45,226-0018,神奈川県横浜市緑区長津田みなみ台**********,S14040,20150708,0-00000000-0
7,CS034515000235,松田 倫子,1,女性,1967-01-23,52,216-0001,神奈川県川崎市宮前区野川**********,S14034,20150924,0-00000000-0
8,CS004315000319,金田 陽子,1,女性,1984-02-23,35,176-0025,東京都練馬区中村南**********,S13004,20160410,2-20090925-1
9,CS004313000062,三輪 美優,1,女性,1985-07-21,33,176-0024,東京都練馬区中村**********,S13004,20151010,0-00000000-0
